In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
def scrape_page_source(driver,url):
    driver.get(url)
    driver.maximize_window()
    driver.implicitly_wait(10)
    # Scroll Down 
    SCROLL_PAUSE_TIME = 0.5
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight") 
    while True:
         # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight") # Calculate new scroll height
        if new_height == last_height:
            break
        last_height = new_height
    return driver.page_source

In [3]:
home_page_url = "https://www.amazon.in/gp/bestsellers/electronics/1805560031"
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))  
page_source = scrape_page_source(driver,home_page_url)
driver.close()

soup = BeautifulSoup(page_source, 'lxml')
domain = "https://www.amazon.in/"

# Extracting website url 
starting_tag = soup.find_all('li',class_="a-selected")
base_url = domain+starting_tag[0].a['href'][:-1]
ending_tag = soup.find_all('li',class_="a-last")
endPageNum = int(ending_tag[0].a['href'][-1])

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.81M/6.81M [00:01<00:00, 4.45MB/s]


In [10]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) 
driver.maximize_window()
product_links = []

for i in range(endPageNum):
    page_link = base_url+str(i+1) 
    page_review = scrape_page_source(driver,page_link)
    product_soup = BeautifulSoup(page_review, 'lxml')
    link_tags = product_soup.find_all('div',id="gridItemRoot")
    for i in range(len(link_tags)):
        product_links.append(domain+link_tags[i].a['href'])
driver.close()

print(len(product_links))

100


## Scrape Reviews

In [5]:
practice_link = product_links[0]      # Practice link
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) # Chrome Driver
driver.get(practice_link)  # GET Request to mentioned link
product_soup = BeautifulSoup(driver.page_source, 'lxml')  # Create soup objct using page source 
driver.close()
tags = product_soup.find_all('a',class_='a-link-emphasis a-text-bold')  # Locate "See all reviews button" on product page
print(domain+tags[0]['href'])  # Link for all reviews

https://www.amazon.in//realme-Oxygen-Storage-Processor-Battery/product-reviews/B09FKDB27R/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


In [6]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) 
driver.get(practice_link)
product_soup = BeautifulSoup(driver.page_source, 'lxml')

In [7]:
tags = product_soup.find_all('a',class_='a-link-emphasis a-text-bold')

In [8]:
tags[0]['href']

'/realme-Oxygen-Storage-Processor-Battery/product-reviews/B09FKDB27R/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'

In [15]:
link = "https://www.amazon.in/Samsung-Storage-Battery-Octa-Core-Processor/product-reviews/B0BZCSNTT4/ref=cm_cr_arp_d_paging_btm_prev_1?ie=UTF8&pageNumber=1&reviewerType=all_reviews"
# link = "https://www.amazon.in/Samsung-Storage-Battery-Octa-Core-Processor/product-reviews/B0BZCSNTT4/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) 
driver.get(link)